In [1]:
pip install schedule

# Import Library

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
from scipy.stats import ks_2samp
from sklearn.model_selection import StratifiedKFold
from io import StringIO
from datetime import datetime
import time
import pytz
from datetime import timedelta
import concurrent.futures
import concurrent.futures
import sys

# Konversi Tanggal

Mengubah data tanggal dengan format sesuai dengan kolom call_time agar python dapat membaca data tersebut sebagi data tanggal.

In [2]:
# mengubah tipe data menjadi tipe datetime
def konversi_tanggal(string_tanggal):
    return datetime.strptime(string_tanggal, "%Y-%m-%d %H:%M:%S")

# Filtering Data

Melakukan filtering terhadap data, sejak data dibaca hingga menghasilkan data yang telah difilter (blacklist, ispstn = 1, outgoing).

In [3]:
# Filtering data
def filtering_data(df):
  # Membersihkan nama kolom
  df.columns = df.columns.str.replace(' revas_frames2_collection_202311.', '')
  df.columns = df.columns.str.strip()

  # Menghilangkan spasi pada data : '  outgoing  ' --> 'outgoing'
  df = df.apply(lambda x: x.str.strip() if x.dtype == 'O' else x)

  # Menghapus data yang mengandung nilai '+-----+'
  nilai_yang_dihapus = '+------------------------------------------------+--------------------------------------------+-------------------------------------------+-----------------------------------------------+---------------------------------------+---------------------------------------+------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+-------------------------------------------+------------------------------------------+--------------------------------------------+--------------------------------------------------+--------------------------------------------------+---------------------------------------------------+---------------------------------------------------+-----------------------------------------------+----------------------------------------------+----------------------------------------------+----------------------------------------------+--------------------------------------------+--------------------------------------------+----------------------------------------------+--------------------------------------------+--------------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+---------------------------------------+------------------------------------------------+------------------------------------------------+--------------------------------------------+--------------------------------------------+-------------------------------------------------+----------------------------------------------------+----------------------------------------------------+-------------------------------------------------+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+---------------------------------------------+---------------------------------------------+----------------------------------------------+----------------------------------------------+-------------------------------------------------+-------------------------------------------------+--------------------------------------------+--------------------------------------------+--------------------------------------------+--------------------------------------------+---------------------------------------------------+------------------------------------------------+-------------------------------------------------+--------------------------------------------------+---------------------------------------------------+-------------------------------------------------+--------------------------------------------------+---------------------------------------------------+-------------------------------------------------+---------------------------------------------------+----------------------------------------------------+-------------------------------------------------+---------------------------------------------------+----------------------------------------------------+----------------------------------------------------+---------------------------------------------+------------------------------------------+-----------------------------------------------+-------------------------------------+'
  df = df[df.iloc[:, 0] != nilai_yang_dihapus]

  # Menghapus kolom yang tidak digunakan
  df = df.drop(df.columns[0], axis=1)
  df = df.drop(df.columns[-1], axis=1)

  # Mengubah tipe data menjadi string
  df = df.astype(str)
  df.reset_index(drop=True, inplace=True)

  # Menghapus data yang mengandung nilai ' revas_frames2_collection_202311.inserted_date  '
  df = df[df['inserted_date'] != ' revas_frames2_collection_202311.inserted_date  ']

  # Filtering data OUTGOING dan BLACLIST
  df = df[df['calltype'] == 'OUTGOING']
  df = df[df['dest_type'] == 'BLACKLIST']

  # Mengubah tipe data menjadi Integer
  df['anum'] = df['anum'].astype(int)
  df['bnum'] = df['bnum'].astype(int)
  df['duration'] = df['duration'].astype(int)
  df['filerow'] = df['filerow'].astype(int)
  df['access_code'] = df['access_code'].astype(int)
  df['dest_prefix'] = df['dest_prefix'].astype(int)
  df['dest_wildcard'] = df['dest_wildcard'].astype(int)
  df['orig_wildcard'] = df['orig_wildcard'].astype(int)
  df['anum_tm88'] = df['anum_tm88'].astype(int)
  df['bnum_tm88'] = df['bnum_tm88'].astype(int)
  df['anum_isgsm'] = df['anum_isgsm'].astype(int)
  df['bnum_isgsm'] = df['bnum_isgsm'].astype(int)
  df['anum_ispstn'] = df['anum_ispstn'].astype(int)
  df['bnum_ispstn'] = df['bnum_ispstn'].astype(int)
  df['anum_isroaming'] = df['anum_isroaming'].astype(int)
  df['bnum_isroaming'] = df['bnum_isroaming'].astype(int)
  df['orig_cost'] = df['orig_cost'].astype(int)
  df['dest_cost'] = df['dest_cost'].astype(int)
  df['orig_anum'] = df['orig_anum'].astype(int)
  df['orig_bnum'] = df['orig_bnum'].astype(int)
  df['ds'] = df['ds'].astype(int)

  # Filtering data ispstn = 1
  df = df[df['anum_ispstn'] == 1]

  # Mengubah tipe data call_time menjadi datetime agar terbaca sebagai tanggal oleh python
  df['call_time'] = df['call_time'].apply(konversi_tanggal)
  df.reset_index(drop=True, inplace=True)
  return df

# Feature Engineer

Melakukan feature engineer untuk memilih beberapa feature yang nantinya akan diproses oleh model.

Terdapat 5 fitur yang dibentuk pada tahap ini.

1.   Duration: Jumlah total durasi call antara anum dan bnum
2.   Count: Jumlah banyaknya call antara anum dan bnum
3.   Total_duration: Jumlah total durasi call pada satu anum
4.   Diff_call: Jumlah bnum yang berbeda oleh satu anum
5.   total_call: jumlah banyaknya call dari satu anum



In [4]:
# Feature Engineer
def feature_engineer(df):

  # Membuat feature duration & count
  df['count'] = 0
  result_baru = pd.DataFrame()

  # Melakukan grouping berdasarkan anum dan bnum yang sama dengan menjumlahkan duration dan menghitung count
  result_baru = df.groupby(['anum', 'bnum']).agg({'duration': 'sum', 'count': 'size'}).reset_index()


  # Membuat feature total_durration & diff_call
  result_baru_new = pd.DataFrame()
  # Melakukan grouping berdasarkan anum dengan menjumlahkan duration sebagai total duration dan menghitung count untuk menghitung diff_call
  result_baru_new = result_baru.groupby('anum').agg({'duration': 'sum', 'count': 'size'}).reset_index()
  result_baru_new.rename(columns={'duration': 'total_duration', 'count' : 'diff_call'}, inplace=True)

  # Menggabungkan feature (duration & count) dengan feature (total_duration & diff_call) pada satu dataframe
  result_inner = pd.DataFrame()
  result_inner = pd.merge(result_baru, result_baru_new, on='anum', how='inner')

  # Membuat feature total_call
  result_baru_new_2 = pd.DataFrame()
  # Melakukan grouping berdasarkan anum dengan menjumlahkan count sebagai total call
  result_baru_new_2 = result_inner.groupby('anum').agg({'count': 'sum'}).reset_index()
  result_baru_new_2.rename(columns={'count': 'total_call'}, inplace=True)

  # Menggabungkan seluruh feature pada satu dataframe
  result_inner_2 = pd.DataFrame()
  result_inner_2 = pd.merge(result_inner, result_baru_new_2, on='anum', how='inner')
  return result_inner_2

# Agregasi

Melakukan agregasi terhadap data yang baru datang untuk diagregasikan dengan data sebelumnya (dengan format data feature engineer)

In [5]:
# Agregasi
def agregasi(df, data): # df adalah data baru, data adalah data sebelumnya
  # Try except untuk menghindari key error
  try:
      df['count'] = 0
      result_baru = pd.DataFrame()
      # Melakukan grouping berdasarkan anum dan bnum yang sama pada dat baru untuk mendapatkan feature duration dan count
      result_baru = df.groupby(['anum', 'bnum']).agg({'duration': 'sum', 'count': 'size'}).reset_index()

      # Melakukan concat atau penggabungan data antara data baru yang sudah di grouping dan dengan data sebelumnya
      result = pd.concat([data, result_baru], axis=0)
      result = result.reset_index()

      # Menghapus kolom yang tidak digunakan
      result = result.drop('Unnamed: 0', axis=1)
      result = result.drop('index', axis=1)

      # Mengikuti langkah-langkah pada feature engineer untuk mendapatkan feature yang sesuai
      result = result.groupby(['anum', 'bnum']).agg({'duration': 'sum', 'count': 'sum'}).reset_index()
      result_baru_new = pd.DataFrame()
      result_baru_new = result.groupby('anum').agg({'duration': 'sum', 'count': 'size'}).reset_index()
      result_baru_new.rename(columns={'duration': 'total_duration', 'count' : 'diff_call'}, inplace=True)
      result_inner = pd.DataFrame()
      result_inner = pd.merge(result, result_baru_new, on='anum', how='inner')
      result_baru_new_2 = pd.DataFrame()
      result_baru_new_2 = result_inner.groupby('anum').agg({'count': 'sum'}).reset_index()
      result_baru_new_2.rename(columns={'count': 'total_call'}, inplace=True)
      result_inner_2 = pd.DataFrame()
      result_inner = pd.merge(result_inner, result_baru_new_2, on='anum', how='inner')
      return result_inner
  except KeyError:
      df['count'] = 0
      result_baru = pd.DataFrame()
      # Melakukan grouping berdasarkan anum dan bnum yang sama pada dat baru untuk mendapatkan feature duration dan count
      result_baru = df.groupby(['anum', 'bnum']).agg({'duration': 'sum', 'count': 'size'}).reset_index()

      # Melakukan concat atau penggabungan data antara data baru yang sudah di grouping dan dengan data sebelumnya
      result = pd.concat([data, result_baru], axis=0)
      result = result.reset_index()

      # Mengikuti langkah-langkah pada feature engineer untuk mendapatkan feature yang sesuai
      result = result.groupby(['anum', 'bnum']).agg({'duration': 'sum', 'count': 'sum'}).reset_index()
      result_baru_new = pd.DataFrame()
      result_baru_new = result.groupby('anum').agg({'duration': 'sum', 'count': 'size'}).reset_index()
      result_baru_new.rename(columns={'duration': 'total_duration', 'count' : 'diff_call'}, inplace=True)
      result_inner = pd.DataFrame()
      result_inner = pd.merge(result, result_baru_new, on='anum', how='inner')
      result_baru_new_2 = pd.DataFrame()
      result_baru_new_2 = result_inner.groupby('anum').agg({'count': 'sum'}).reset_index()
      result_baru_new_2.rename(columns={'count': 'total_call'}, inplace=True)
      result_inner_2 = pd.DataFrame()
      result_inner = pd.merge(result_inner, result_baru_new_2, on='anum', how='inner')
      return result_inner

# Fungsi Pisah Tanggal


Fungsi untuk memisahkan data hari ini dengan data hari kemarin

In [6]:
def pisah_tanggal(df):
  tanggal_hari_ini = datetime(2023, 11, 28)
  sehari_sebelum = tanggal_hari_ini - timedelta(days=1)
  df_hari_ini = df[df['call_time'].dt.date == tanggal_hari_ini.date()]
  df_kemarin = df[df['call_time'].dt.date == sehari_sebelum.date()]

  return df_hari_ini, df_kemarin

# Fungsi Reset Dataframe

Fungsi untuk melakukan reset data frame, mengganti df_kemarin dengan df_hari_ni dan mereset df_hari_ini

In [7]:
def reset_df():
    global df_hari_ini
    global df_kemarin
    # Lakukan reset dan penggantian nilai df_hari_ini dengan df_kemarin
    df_kemarin = df_hari_ini
    df_hari_ini = pd.DataFrame()

# Fungsi Predict menggunakan model


Model menggunakan decisiontree

In [8]:
from joblib import load

In [9]:
model = load('rf_model.joblib')

In [10]:
def model_predict(model, df):
  X = df.iloc[:, 2:7].values
  Y_pred = model.predict(X)
  df['is_fraud'] = Y_pred

  return df

# Membaca data dari csv

In [11]:
nama_file = 'blacklist28.csv'
df = pd.read_csv(nama_file, sep='|', skiprows=1)

# Eksekusi pipeline secara paralel

Mengeksekusi kedua pipeline yang berbeda secara paralel.

1.   Pipeline1 mengeksekusi pipeline untuk data hari ini
2.   Pipeline2 mengeksekusi pipeline untuk data hari kemarin



In [12]:
# Fungsi pipeline1
def pipeline1(data):
  global df_hari_ini
  #Filtering data
  data = filtering_data(data)
  #Memisahkan data berdasarkan tanggal
  df_today, df_yesterday = pisah_tanggal(data)
  try:
    # Jika terdapat data sebelumnya, agregasi data baru dengan data sebelumnya
    df_model = agregasi(df_today,df_hari_ini)
    df_model = model_predict(model, df_model)
    return df_model
  except NameError:
    # Jika tidak terdapat data sebelumnya, akan dilakukan feature engineer
    df_model = feature_engineer(df_today)
    df_model = model_predict(model, df_model)
    return df_model

def pipeline2(data):
  global df_kemarin
  #Filtering data
  data = filtering_data(data)
  #Memisahkan data berdasarkan tanggal
  df_today, df_yesterday = pisah_tanggal(data)
  try:
    # Jika terdapat data sebelumnya, agregasi data baru dengan data sebelumnya
    df_model = agregasi(df_yesterday,df_kemarin)
    df_model = model_predict(model, df_model)
    return df_model
  except NameError:
    # Jika tidak terdapat data sebelumnya, akan dilakukan feature engineer
    df_model = feature_engineer(df_yesterday)
    df_model = model_predict(model, df_model)
    return df_model

# Main program
if __name__ == "__main__":
    # Mendefinisikan waktu yang digunakan adalah jakara
    jakarta_timezone = pytz.timezone('Asia/Jakarta')

    # Dapatkan waktu saat ini di Jakarta
    waktu_sekarang_jakarta = datetime.now(jakarta_timezone)

    if waktu_sekarang_jakarta.hour == 0 and 0 <= waktu_sekarang_jakarta.minute <= 5:
        reset_df()
        sys.exit()

    data_to_process = [df]

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Eksekusi pipeline1(data) secara paralel
        future1 = executor.submit(pipeline1, df)

        # Eksekusi pipeline2(data) secara paralel
        future2 = executor.submit(pipeline2, df)

        # Tunggu kedua pipeline selesai
        concurrent.futures.wait([future1, future2])

        # Ambil hasil jika diperlukan
        result1 = future1.result()
        result2 = future2.result()

        df_hari_ini = result1
        df_kemarin = result2

    # Lakukan sesuatu dengan hasil kedua pipeline

    df_kemarin


<ipython-input-3-3663c047a570>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(' revas_frames2_collection_202311.', '')
<ipython-input-3-3663c047a570>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(' revas_frames2_collection_202311.', '')


In [13]:
df_kemarin

,anum,bnum,duration,count,total_duration,diff_call,total_call,is_fraud
0,2150805447,96892176433,20,1,20,1,1,0
1,2180861000,21651095269,22,1,97,7,8,0
2,2180861000,26772351750,6,2,97,7,8,0
3,2180861000,50581819115,16,1,97,7,8,0
4,2180861000,50584151306,6,1,97,7,8,0
5,2180861000,50585225649,23,1,97,7,8,0
6,2180861000,881623418927,16,1,97,7,8,0
7,2180861000,881677109455,8,1,97,7,8,0


In [14]:
df_hari_ini

,anum,bnum,duration,count,total_duration,diff_call,total_call,is_fraud
0,76195441,358505554629,488,1,488,1,1,0
1,211071077,4533434200,149,1,12741,13,38,0
2,211071077,31202455355,280,4,12741,13,38,0
3,211071077,31713646200,2047,11,12741,13,38,0
4,211071077,35724030270,109,3,12741,13,38,0
...,...,...,...,...,...,...,...,...
321,6188819888,41443607070,1050,3,1526,4,12,0
322,6188819888,41818366000,141,4,1526,4,12,0
323,7784888898,302104293843,222,1,222,1,1,0
324,7785706066,870776654027,16,2,16,1,2,0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_kemarin

,anum,bnum,duration,count,total_duration,diff_call,total_call
0,76195441,358505554629,1952,4,1952,1,4
1,211071077,4533434200,596,4,50964,13,152
2,211071077,31202455355,1120,16,50964,13,152
3,211071077,31713646200,8188,44,50964,13,152
4,211071077,35724030270,436,12,50964,13,152
...,...,...,...,...,...,...,...
321,6188819888,41443607070,4200,12,6104,4,48
322,6188819888,41818366000,564,16,6104,4,48
323,7784888898,302104293843,888,4,888,1,4
324,7785706066,870776654027,64,8,64,1,8
